# Data Generation Example

In [1]:
import sys
import numpy as np
sys.path.append("../")

from data_loader import CityScapesNumpy, plot_detections
from tracking.tracking_utils import iou_2d
from scipy.optimize import linear_sum_assignment

# ...

In [2]:
ASSIGNMENT_THRESH = 0.0

In [8]:
class Track:
    def __init__(self, id, label):
        self.id = id
        self.label = label
        self.dets = []
        
    def add_det(self, det):
        self.dets.append(det)
        
    def get_last_det(self):
        return self.dets[-1]
        

In [4]:
def get_track(idd, tracks):
    for track in tracks:
        if track.id == idd:
            return track
        
def get_track_idx(idd, tracks):
    for i, track in enumerate(tracks):
        if track.id == idd:
            return i

In [5]:
data_set = CityScapesNumpy()

  0%|▎                                                                                 | 2/599 [00:00<00:35, 16.92it/s]

Loading data into array...


100%|████████████████████████████████████████████████████████████████████████████████| 599/599 [00:32<00:00, 18.64it/s]


Size of data:  2709847248


In [17]:
tracks = []
cur_id = 0
tracking_category = 3 # class car

for i in range(len(data_set)):
    # TODO: filter class
    if tracks:
        old_boxes = [track.get_last_det()for track in tracks]
        old_ids = [track.id for track in tracks] 
    else:
        old_boxes = []
        
    img_next, boxes_next, lbls_next, scrs_next = data_set.get_idx(i)
    
    category_mask = lbls_next == tracking_category
    boxes_next, lbls_next, scrs_next = boxes_next[category_mask], lbls_next[category_mask], scrs_next[category_mask]
    
    pair_cost = np.array([[iou_2d(bx1, bx2) for bx1 in old_boxes] for bx2 in boxes_next])
    pair_cost *= -1
    row_ind, col_ind = linear_sum_assignment(pair_cost)
        
    pairs = [(row_ind[j], col_ind[j]) for j, _ in enumerate(row_ind) 
                 if  pair_cost[row_ind[j]][col_ind[j]] < ASSIGNMENT_THRESH]
    
    unmatched_dets = [j for j, _ in enumerate(lbls_next) if j not in col_ind]
    unmatched_hyps = [j for j in range(len(old_boxes)) if j not in row_ind]
    
    print(len(tracks), len(lbls_next))
    print("pairs:", pairs)
    print("unmatched dets:", unmatched_dets)
    print("unmatched hyps:", unmatched_hyps)
    
    # Kill unmatched hyps
    for hyp_idx in unmatched_hyps:
        track_idx = get_track_idx(old_ids[hyp_idx], tracks)
        tracks.pop(track_idx)
    
    # Spawn unmatched detections
    for det_idx in unmatched_dets:
        new_track = Track(cur_id, lbls_next[det_idx])
        new_track.add_det(boxes_next[det_idx])
        cur_id += 1
        tracks.append(new_track)
        
    # Update pairs
    for (det_idx, hyp_idx) in pairs:
        track = get_track(old_ids[hyp_idx], tracks)
        track.add_det(boxes_next[det_idx])
    
    print(tracks)
    

0 10
pairs: []
unmatched dets: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
unmatched hyps: []
[<__main__.Track object at 0x000002F12F6FAE08>, <__main__.Track object at 0x000002F1002A8108>, <__main__.Track object at 0x000002F12F6FAEC8>, <__main__.Track object at 0x000002F12F6FAE48>, <__main__.Track object at 0x000002F12F6FABC8>, <__main__.Track object at 0x000002F12F6FAD88>, <__main__.Track object at 0x000002F12F6FAF48>, <__main__.Track object at 0x000002F12F6FADC8>, <__main__.Track object at 0x000002F12F6FAF88>, <__main__.Track object at 0x000002F12F6FAD48>]
10 7
pairs: [(0, 0), (1, 2), (2, 1), (3, 3), (4, 4), (5, 6), (6, 8)]
unmatched dets: [5]
unmatched hyps: [7, 8, 9]


AttributeError: 'NoneType' object has no attribute 'add_det'

In [ ]:
for i in range(len(data_set)):
    img, bx, lbl, scrs = data_set.get_idx(i)
    plot_detections(img, bx, lbl, scrs, categories=[1, 2, 3, 4, 5, 6, 7, 8, 8, 10])